In [19]:
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF
from scipy.spatial.distance import pdist, cdist, squareform
import numpy as np

X, y = make_friedman2(n_samples=500, noise=0, random_state=0)
kernel = RBF() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
         random_state=0).fit(X, y)


In [13]:
gpr.score(X, y)

-1.439871997881387

In [21]:
print(gpr.kernel_(X).shape, gpr.kernel_)#K = self.kernel_(self.X_train_)
print (X.shape, gpr.X_train_.shape)

dists = pdist(X / 1, metric='sqeuclidean')
K = np.exp(-.5 * dists)
print(K.shape)
# convert from upper-triangular matrix to square matrix
K = squareform(K)
print(K.shape)
np.fill_diagonal(K, 1)


(500, 500) RBF(length_scale=1e+05) + WhiteKernel(noise_level=1e+05)
(500, 4) (500, 4)
(124750,)
(500, 500)
